In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence, pack_sequence
import onnx

from model_model import SiameseNetwork
from model_configs import ModelDimConfigs, TrainingConfigs
from misc_tools import get_timestamp

In [2]:
dimconf = ModelDimConfigs(
        rnn_in_size=64, 
        lin_in_size_1=32, 
        lin_in_size_2=16, 
        lin_out_size_2=1
    )

model = SiameseNetwork(
    dimconf=dimconf
)

In [3]:
model

SiameseNetwork(
  (rnn): SelfPackLSTM(
    (rnn): LSTM(64, 32, num_layers=2, batch_first=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
  (sigmoid): Sigmoid()
)

In [6]:
x1 = torch.rand(3, 20, 64)
x2 = torch.rand(3, 12, 64)

l1 = torch.randint(1, 20, (3, ))
l2 = torch.randint(1, 12, (3, ))

In [7]:
o = model((x1, x2), (l1, l2))

In [8]:
onnx_path = "save.onnx"
torch.onnx.export(model, ((x1, x2), (l1, l2)), onnx_path)

In [3]:
torch.save(model.state_dict(), "save.pth")

In [49]:
x = torch.randn(3, 4, 5)
x_lens = torch.tensor([1, 2, 4])

In [51]:
torch.nn.functional.one_hot(x_lens - 1)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]])

In [44]:
nx = x[, :]

In [45]:
nx.shape

torch.Size([3, 3, 4, 5])

In [55]:
torch.index_select(x, 1, x_lens - 1).shape

torch.Size([3, 3, 5])